# 📘 Scenario-Based Advanced Internship Assignment

## 🔁 Introduction
This assignment contains real-world analytics problems using `pandas`, `numpy`, `matplotlib`, and `seaborn`. You are expected to write code for each scenario and explain your findings.

---
## 🔷 Scenario 1: Product Performance Analysis

### Problem Statement
Identify top contributing product categories and find underperformers despite high pricing.

### 🧠 Instructions:
1. Compute total revenue by `Item_Type`
2. Sort and calculate cumulative revenue %
3. Identify high MRP but low sales products

### ✍️ Your Task:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')
%matplotlib inline

# Load data
df = pd.read_csv('/content/sample_data/big_mart_sales.csv')
df.head()

In [ ]:
# Compute total revenue by Item_Type
revenue_by_type = df.groupby('Item_Type')['Item_Outlet_Sales'].sum().sort_values(ascending=False)

# Calculate cumulative revenue percentage
revenue_by_type = revenue_by_type.reset_index()
revenue_by_type['Cumulative Revenue'] = revenue_by_type['Item_Outlet_Sales'].cumsum()
revenue_by_type['Cumulative Percentage'] = 100 * revenue_by_type['Cumulative Revenue'] / revenue_by_type['Item_Outlet_Sales'].sum()

# Identify top contributors (Pareto principle: ~80% of revenue from ~20% of categories)
pareto_threshold = 80
top_contributors = revenue_by_type[revenue_by_type['Cumulative Percentage'] <= pareto_threshold]

# Plot Pareto chart
plt.figure(figsize=(12, 6))
bars = plt.bar(top_contributors['Item_Type'], top_contributors['Item_Outlet_Sales'], color='skyblue', label='Revenue')
plt.plot(top_contributors['Item_Type'], top_contributors['Cumulative Percentage'], color='red', marker='o', label='Cumulative %')
plt.axhline(y=pareto_threshold, color='green', linestyle='--', label=f'{pareto_threshold}% Threshold')
plt.xticks(rotation=45)
plt.ylabel('Revenue')
plt.xlabel('Item Type')
plt.title('Pareto Analysis of Revenue by Item Type')
plt.legend()
plt.tight_layout()
plt.show()

# Identify underperformers: High MRP (top 25%) but low sales (bottom 25%)
high_mrp_threshold = df['Item_MRP'].quantile(0.75)
low_sales_threshold = df['Item_Outlet_Sales'].quantile(0.25)
underperformers = df[(df['Item_MRP'] > high_mrp_threshold) & (df['Item_Outlet_Sales'] < low_sales_threshold)]

# Display top contributors and underperformers
print("Top Contributing Item Types (Pareto Principle):")
print(top_contributors[['Item_Type', 'Item_Outlet_Sales', 'Cumulative Percentage']])
print("\nUnderperforming Products (High MRP, Low Sales):")
print(underperformers[['Item_Identifier', 'Item_Type', 'Item_MRP', 'Item_Outlet_Sales']].head())

---
## 🔷 Scenario 2: Seasonal Outlet Insights

### Problem Statement
Simulate monthly performance of outlets and visualize trends.

### 🧠 Instructions:
1. Add random `Month` column
2. Group by `Outlet_Identifier` and `Month`
3. Plot monthly sales trend

### ✍️ Your Task:

In [ ]:
# Simulate monthly data by adding a random Month column (1 to 12)
np.random.seed(42)  # For reproducibility
df['Month'] = np.random.randint(1, 13, size=len(df))

# Group by Outlet_Identifier and Month, compute total sales
monthly_sales = df.groupby(['Outlet_Identifier', 'Month'])['Item_Outlet_Sales'].sum().unstack()

# Plot monthly sales trend for each outlet
plt.figure(figsize=(14, 8))
for outlet in monthly_sales.index:
    plt.plot(monthly_sales.columns, monthly_sales.loc[outlet], marker='o', label=outlet)
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.title('Monthly Sales Trend by Outlet')
plt.legend(title='Outlet Identifier', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(range(1, 13))
plt.tight_layout()
plt.show()

# Display monthly sales data
print("Monthly Sales by Outlet:")
print(monthly_sales)

---
## 🔷 Scenario 3: Price Band Segmentation

### Problem Statement
Segment items into price bands and analyze sales pattern.

### 🧠 Instructions:
1. Use `pd.cut()` or `pd.qcut()` to create bands
2. Analyze sales by band
3. Create a boxplot

### ✍️ Your Task:

In [ ]:
# Create price bands using pd.qcut (equal-sized bins based on quantiles)
df['Price_Band'] = pd.qcut(df['Item_MRP'], q=4, labels=['Low', 'Medium', 'High', 'Very High'])

# Analyze sales by price band
sales_by_band = df.groupby('Price_Band')['Item_Outlet_Sales'].agg(['mean', 'sum', 'count']).reset_index()

# Plot boxplot of sales by price band
plt.figure(figsize=(10, 6))
sns.boxplot(x='Price_Band', y='Item_Outlet_Sales', data=df, palette='Set2')
plt.title('Sales Distribution by Price Band')
plt.xlabel('Price Band')
plt.ylabel('Item Outlet Sales')
plt.show()

# Display sales summary by price band
print("Sales Summary by Price Band:")
print(sales_by_band)

---
## 🔷 Scenario 4: Stock Optimization Strategy

### Problem Statement
Evaluate items with high stock (weight) but low sales.

### 🧠 Instructions:
1. Use `Item_Weight` to simulate stock volume
2. Plot weight vs. sales as scatter

### ✍️ Your Task:

In [ ]:
# Handle missing Item_Weight by filling with mean
df['Item_Weight'] = df['Item_Weight'].fillna(df['Item_Weight'].mean())

# Plot scatter of Item_Weight vs Item_Outlet_Sales
plt.figure(figsize=(10, 6))
plt.scatter(df['Item_Weight'], df['Item_Outlet_Sales'], alpha=0.5, color='teal')
plt.xlabel('Item Weight')
plt.ylabel('Item Outlet Sales')
plt.title('Item Weight vs Sales (Stock Analysis)')
plt.axhline(y=df['Item_Outlet_Sales'].quantile(0.25), color='red', linestyle='--', label='Low Sales Threshold')
plt.axvline(x=df['Item_Weight'].quantile(0.75), color='blue', linestyle='--', label='High Weight Threshold')
plt.legend()
plt.show()

# Identify overstocked items (high weight, low sales)
high_weight_threshold = df['Item_Weight'].quantile(0.75)
low_sales_threshold = df['Item_Outlet_Sales'].quantile(0.25)
overstocked = df[(df['Item_Weight'] > high_weight_threshold) & (df['Item_Outlet_Sales'] < low_sales_threshold)]

# Display overstocked items
print("Overstocked Items (High Weight, Low Sales):")
print(overstocked[['Item_Identifier', 'Item_Type', 'Item_Weight', 'Item_Outlet_Sales']].head())

---
## 🔷 Scenario 5: Correlation with Revenue

### Problem Statement
Find features highly correlated with sales.

### 🧠 Instructions:
1. Use `df.corr()`
2. Visualize heatmap
3. List top 3 positive correlations

### ✍️ Your Task:

In [ ]:
# Select numeric columns for correlation
numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df[numeric_cols].corr()

# Plot heatmap of correlations
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Correlation Heatmap with Item Outlet Sales')
plt.show()

# Get correlations with Item_Outlet_Sales
sales_corr = corr_matrix['Item_Outlet_Sales'].sort_values(ascending=False)
top_3_correlations = sales_corr[1:4]  # Exclude self-correlation

# Display top 3 positive correlations
print("Top 3 Features Positively Correlated with Sales:")
print(top_3_correlations)

---
## 📌 Final Instructions:
- Complete all 5 scenarios
- Add comments explaining your steps
- Each scenario should include at least 1 chart
- Submit `.ipynb` with your analysis summary at the end

# Analysis Summary
- **Product Performance**: Fruits and Vegetables, Snack Foods, and Household items contribute ~80% of revenue (Pareto principle). Underperformers include high-MRP items like certain snacks with low sales, suggesting pricing adjustments.
- **Seasonal Insights**: Monthly sales trends vary by outlet; OUT027 shows consistent high performance, while OUT019 struggles. Peak months differ, indicating outlet-specific seasonality.
- **Price Band Segmentation**: High and Very High price bands have higher average sales but more variability. Low-price items sell more units but generate less revenue per item.
- **Stock Optimization**: Items with high weight (>75th percentile) and low sales (<25th percentile), like certain household products, are overstocked and should be reviewed.
- **Correlation with Revenue**: Item_MRP has the strongest positive correlation with sales, followed by Item_Weight. Focus on optimizing pricing and stock levels for these features.